In [12]:
# ------------------------------
# 0️1 Install dependencies (if not installed)
# ------------------------------
!pip install -U groq openai langchain langchain-openai langchain-community

# ------------------------------
# 02 Streamlit: For building the web app frontend
# ------------------------------
!pip install -q streamlit

# ------------------------------
# 03 pyngrok: To expose Colab port to a public URL
# ------------------------------
!pip install -q pyngrok

# ------------------------------
# 04 Optional / Useful Packages:
# ------------------------------
# python-dotenv: Load API keys from a .env file instead of hardcoding
# jsonschema: Validate extracted JSON output
!pip install -q python-dotenv jsonschema

In [13]:
from google.colab import userdata
api_key = userdata.get('GROQ_API_KEY')

In [14]:
# ===============================================
# Conversation Management & JSON Extraction using Groq API
# ===============================================

import os
import json
import requests

# ------------------------------
# 1. Setup your API credentials
# ------------------------------
# Use your Groq API key here (looks like gsk_...)
API_KEY = api_key   # <-- replace with actual key string
BASE_URL = "https://api.groq.com/openai/v1/chat/completions"
HEADERS = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

# ===============================================
# Task 1: Conversation Management & Summarization
# ===============================================

# We'll keep everything in memory (simple dictionary).
# Each session_id will have its own conversation history.
conversations = {}
message_counter = {}
PERIODIC_K = 3   # after how many turns should we summarize


def chat_request(messages, model="llama-3.1-8b-instant", temperature=0.0):
    """
    Send a list of messages to Groq API and get back the assistant reply.
    Messages must follow the format:
    [{"role": "user"/"assistant"/"system", "content": "..."}]
    """
    payload = {"model": model, "messages": messages, "temperature": temperature}
    response = requests.post(BASE_URL, headers=HEADERS, json=payload)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]


def get_history(session_id):
    """
    Grab the conversation history for this user/session.
    If it's the first time we see this session, start fresh.
    """
    if session_id not in conversations:
        conversations[session_id] = []
        message_counter[session_id] = 0
    return conversations[session_id]


def add_message(session_id, role, content):
    """
    Add one message to the running history.
    Example:
      add_message("user123", "user", "Hello there")
    """
    history = get_history(session_id)
    history.append({"role": role, "content": content})


def summarize_history(session_id):
    """
    Take the entire chat so far and ask the model to summarize it.
    This helps keep a shorter memory when chats get too long.
    """
    history = get_history(session_id)
    text_history = "\n".join([f"{m['role']}: {m['content']}" for m in history])
    summary_prompt = [
        {"role": "system", "content": "Summarize the following conversation:"},
        {"role": "user", "content": text_history}
    ]
    return chat_request(summary_prompt)


def converse(session_id, user_input):
    """
    Handle one user input:
    - add it to history
    - get AI reply
    - store reply back into history
    - every few turns, also get a summary
    """
    # user speaks
    add_message(session_id, "user", user_input)

    # send whole history to model
    history = get_history(session_id)
    response = chat_request(history)

    # model speaks
    add_message(session_id, "assistant", response)

    # update turn counter
    message_counter[session_id] += 1

    # sometimes we summarize
    summary = None
    if message_counter[session_id] % PERIODIC_K == 0:
        summary = summarize_history(session_id)

    return response, summary


# ---- Example Run (Task 1) ----
print("\n=== Task 1: Conversation Management ===")
session = "user123"

# Imagine a little back-and-forth chat
user_msgs = [
    "Hello, my name is Nazim.",
    "I belong to Delhi.",
    "I am interested in Artificial Intelligence.",
    "I also like cricket.",
    "What do you know about GPT models?"
]

# Feed the chat step by step
for msg in user_msgs:
    reply, summary = converse(session, msg)
    print(f"\nUser: {msg}\nAI: {reply}")
    if summary:
        print(f"\n---- Summary ----\n{summary}")



=== Task 1: Conversation Management ===

User: Hello, my name is Nazim.
AI: Hello Nazim, what can I help you with today?

User: I belong to Delhi.
AI: Delhi is a beautiful city with a rich history and culture. Home to the Mughal Empire, the British Raj, and a vibrant modern city, Delhi has so much to offer. From the iconic Red Fort to the bustling markets of Chandni Chowk, there's always something new to discover. What's your favorite thing about Delhi, Nazim?

User: I am interested in Artificial Intelligence.
AI: There are some top-notch AI labs and research institutions in India, and Delhi is home to many of them. The Indian Institute of Technology (IIT) Delhi, for example, has a thriving AI department with various research projects and collaborations.

Additionally, companies like HCL Technologies, Wipro, and Infosys have a significant presence in Delhi, working on AI and machine learning projects.

Are you an aspiring AI researcher, developer, or perhaps an undergraduate student l

In [15]:
# ===============================================
# Task 2: JSON Schema Extraction
# ===============================================

# Here we define the structure we expect to pull from free text.
# The model will try to fill in these fields.
json_schema = {
    "name": "string",
    "email": "string",
    "phone": "string",
    "location": "string",
    "age": "integer"
}


def extract_info(chat_text):
    """
    Take a chunk of user text and try to pull structured info out of it.
    Example:
      Input: "Hi, I'm Sara from Bangalore. Email: sara123@gmail.com"
      Output: { "name": "Sara", "email": "sara123@gmail.com", "phone": null, ... }
    """
    # We ask the model very explicitly:
    # - Only output JSON
    # - Stick to the schema
    # - If something's missing, set it as null
    system_prompt = f"""
    Extract the following details from the user chat in JSON format (no Python, no extra text):
    {json.dumps(json_schema, indent=2)}
    If a field is missing, return it as null.
    """

    # Build the conversation we send to the model
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": chat_text}
    ]

    # Ask Groq model for completion
    response = chat_request(messages)

    # Try to parse the output as JSON directly
    try:
        return json.loads(response)
    except:
        # If model replies with something messy (not valid JSON),
        # just return the raw text so we can debug.
        return {"raw_output": response}


# ---- Example Run (Task 2) ----
print("\n=== Task 2: JSON Extraction ===")

# A few test cases to see how well it extracts info
examples = [
    "Hello, I'm Sara from Bangalore. Email: sara123@gmail.com, phone: 9123456789. Age 28.",
    "Hi, this is John from Mumbai. Age 32. Email: john_doe@hotmail.com.",
    "My name is Aditi. Phone: 9876543210."
]

# Try each one
for ex in examples:
    print(f"\nInput: {ex}")
    print("Extracted JSON:", extract_info(ex))



=== Task 2: JSON Extraction ===

Input: Hello, I'm Sara from Bangalore. Email: sara123@gmail.com, phone: 9123456789. Age 28.
Extracted JSON: {'name': 'Sara', 'email': 'sara123@gmail.com', 'phone': '9123456789', 'location': 'Bangalore', 'age': 28}

Input: Hi, this is John from Mumbai. Age 32. Email: john_doe@hotmail.com.
Extracted JSON: {'name': 'John', 'email': 'john_doe@hotmail.com', 'phone': None, 'location': 'Mumbai', 'age': 32}

Input: My name is Aditi. Phone: 9876543210.
Extracted JSON: {'name': 'Aditi', 'email': None, 'phone': '9876543210', 'location': None, 'age': None}


In [16]:
# ===============================================
# Conversation Management & JSON Extraction using Groq API
# (Streamlit version without LangChain)
# ===============================================

import os
import json
import requests
import streamlit as st

# ------------------------------
# 1️⃣ Credentials
# ------------------------------
API_KEY = "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"   # <-- Replace with actual
BASE_URL = "https://api.groq.com/openai/v1/chat/completions"
HEADERS = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

# ------------------------------
# 2️⃣ Core Chat Request
# ------------------------------
def chat_request(messages, model="llama-3.1-8b-instant", temperature=0.0):
    """Send chat completion request to Groq API"""
    payload = {"model": model, "messages": messages, "temperature": temperature}
    response = requests.post(BASE_URL, headers=HEADERS, json=payload)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]

# ------------------------------
# 3️⃣ Session State Initialization
# ------------------------------
if "conversations" not in st.session_state:
    st.session_state.conversations = {}   # {session_id: [messages]}
if "message_counter" not in st.session_state:
    st.session_state.message_counter = {}

PERIODIC_K = 3   # summarize every 3 turns

# ------------------------------
# 4️⃣ Helpers
# ------------------------------
def get_history(session_id):
    if session_id not in st.session_state.conversations:
        st.session_state.conversations[session_id] = []
        st.session_state.message_counter[session_id] = 0
    return st.session_state.conversations[session_id]

def add_message(session_id, role, content):
    history = get_history(session_id)
    history.append({"role": role, "content": content})

def summarize_history(session_id):
    history = get_history(session_id)
    text_history = "\n".join([f"{m['role']}: {m['content']}" for m in history])
    summary_prompt = [
        {"role": "system", "content": "Summarize the following conversation briefly:"},
        {"role": "user", "content": text_history}
    ]
    return chat_request(summary_prompt)

def converse(session_id, user_input):
    add_message(session_id, "user", user_input)
    history = get_history(session_id)
    response = chat_request(history)
    add_message(session_id, "assistant", response)

    st.session_state.message_counter[session_id] += 1
    summary = None
    if st.session_state.message_counter[session_id] % PERIODIC_K == 0:
        summary = summarize_history(session_id)
    return response, summary

# ------------------------------
# 5️⃣ JSON Extraction (Part 2)
# ------------------------------
json_schema = {
    "name": "string",
    "email": "string",
    "phone": "string",
    "location": "string",
    "age": "integer"
}

def extract_info(chat_text):
    system_prompt = f"""
    Extract the following details from the user chat in JSON format and don't give python scripts or any other way:
    {json.dumps(json_schema, indent=2)}
    If a field is missing, return it as null.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": chat_text}
    ]
    response = chat_request(messages)
    try:
        return json.loads(response)
    except:
        return {"raw_output": response}

# ===============================================
# 6️⃣ Streamlit UI
# ===============================================

st.title("🧑‍💻 AI/ML Developer Internship Assignment")
st.write("Conversation Management & JSON Extraction using Groq API (No LangChain)")

# Sidebar for mode
mode = st.sidebar.radio("Select Mode", ["Conversation", "JSON Extraction"])
session_id = st.sidebar.text_input("Session ID", value="user123")

if mode == "Conversation":
    st.subheader("💬 Conversation with Periodic Summarization")

    user_input = st.text_input("Your message:")
    if st.button("Send") and user_input:
        reply, summary = converse(session_id, user_input)

        st.markdown(f"**User:** {user_input}")
        st.markdown(f"**AI:** {reply}")

        if summary:
            st.info(f"📌 Summary after {st.session_state.message_counter[session_id]} turns:\n{summary}")

    st.markdown("### Conversation History")
    for msg in get_history(session_id):
        role = "👤" if msg["role"] == "user" else "🤖"
        st.write(f"{role}: {msg['content']}")

elif mode == "JSON Extraction":
    st.subheader("📊 Extract Info in JSON Format")

    raw_text = st.text_area("Enter text with personal info:")
    if st.button("Extract"):
        extracted = extract_info(raw_text)
        st.json(extracted)


2025-09-25 15:44:12.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 15:44:12.100 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 15:44:12.103 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 15:44:12.104 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 15:44:12.105 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 15:44:12.107 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 15:44:12.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-25 15:44:12.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [17]:
from pyngrok import ngrok
ngrok.set_auth_token("336OOIhxlnjHOZ6gIX3SoT8Jiq4_6zDcAtbp6nQ6Cxa5qJfzW")
ngrok.kill()
public_url = ngrok.connect(8501)
print("🚀 Streamlit Public URL:", public_url)
!streamlit run app.py &


🚀 Streamlit Public URL: NgrokTunnel: "https://unauctioned-millie-preternaturally.ngrok-free.dev" -> "http://localhost:8501"





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.102.108:8501



  Stopping...


## With Langchain Framework ##

In [18]:
# ===============================================
# AI/ML Developer Internship Assignment
# Conversation Management & JSON Extraction using Groq API
# ===============================================

# ------------------------------
# 1️⃣ Imports & Credentials
# ------------------------------
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory, ConversationBufferWindowMemory

# Set Groq / OpenAI API credentials
os.environ["OPENAI_API_KEY"] = api_key   # Replace with your Groq key
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# ------------------------------
# 2️⃣ Initialize LLMs
# ------------------------------
# Main conversational LLM
llm = ChatOpenAI(model="llama-3.1-8b-instant", temperature=0.0)

# LLM for summarization
summarizer_llm = ChatOpenAI(model="llama-3.1-8b-instant", temperature=0.0)

# LLM for structured extraction (Part 2)
llm_extractor = ChatOpenAI(model="llama-3.1-8b-instant", temperature=0)

# ===============================================
# Part 1: Conversation Management & Summarization
# ===============================================

# ------------------------------
# 1.1 Summary Memory Example
# ------------------------------
summary_memory = ConversationSummaryMemory(llm=llm)

conversation_summary = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=True
)

# Example conversation
conversation_summary.predict(input="Hello, my name is Nazim.")
conversation_summary.predict(input="I belong to Delhi.")
conversation_summary.predict(input="I am interested in Artificial Intelligence.")

print("\n=== Summary Memory Buffer ===")
print(summary_memory.buffer)

# ------------------------------
# 1.2 RunnableWithMessageHistory + Session-based conversation
# ------------------------------
store = {}

def get_history(session_id: str) -> ChatMessageHistory:
    """Return chat history for a given session; create new if doesn't exist"""
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Prompt template with history placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "The following is a friendly conversation. Keep track of important details."),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])

conversation_chain = prompt | llm

# Wrap with RunnableWithMessageHistory to handle session-based history
conversation = RunnableWithMessageHistory(
    conversation_chain,
    get_history,
    input_messages_key="input",
    history_messages_key="history"
)

# Helper to summarize a session
def summarize_history(session_id: str):
    """Summarize chat history using summarizer LLM"""
    history = get_history(session_id).messages
    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", "Summarize the following conversation so far."),
        ("human", "{history}")
    ])
    chain = summary_prompt | summarizer_llm
    return chain.invoke({"history": "\n".join([f"{m.type}: {m.content}" for m in history])}).content

# Example conversation
config_user = {"configurable": {"session_id": "user123"}}
conversation.invoke({"input": "Hello there, my name is Nazim."}, config=config_user)
conversation.invoke({"input": "I belong to Delhi."}, config=config_user)
conversation.invoke({"input": "I am interested in Artificial Intelligence."}, config=config_user)

# Show raw history
print("\n---- Conversation History ----")
for msg in get_history("user123").messages:
    print(f"{msg.type}: {msg.content}")

# Show summarized conversation
print("\n---- Summarized Conversation ----")
print(summarize_history("user123"))

# ------------------------------
# 1.3 Window Memory Example (Last 3 Turns)
# ------------------------------
window_memory = ConversationBufferWindowMemory(k=3)

conversation_window = ConversationChain(
    llm=llm,
    memory=window_memory,
    verbose=True
)

conversation_window.predict(input="Hello, I am Sara.")
conversation_window.predict(input="What is the weather today?")
conversation_window.predict(input="Also tell me a joke.")
conversation_window.predict(input="Can you explain quantum physics?")

print("\n---- Last 3 Messages in Window Memory ----")
print(window_memory.buffer)

# ------------------------------
# 1.4 Periodic Summarization after every k-th turn
# ------------------------------
PERIODIC_K = 3  # summarize every 3 turns

def conversation_with_periodic_summary(session_id: str, messages: list):
    """
    Handles conversation and summarizes every k-th turn
    """
    for i, msg in enumerate(messages, 1):
        conversation.invoke({"input": msg}, config={"configurable": {"session_id": session_id}})
        if i % PERIODIC_K == 0:
            summary = summarize_history(session_id)
            print(f"\n---- Summary after {i} turns ----")
            print(summary)

# Test periodic summarization
test_msgs = [
    "Hello, I am Alex.",
    "I live in Mumbai.",
    "I work as a software developer.",
    "I like machine learning.",
    "I also enjoy AI research.",
    "What do you know about GPT models?"
]

conversation_with_periodic_summary("alex123", test_msgs)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, my name is Nazim.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces themselves as Nazim and is greeted by Nova, the AI model. Nova explains its nature as a large language model trained on 45 billion parameters across various topics and sources, including books, articles, and websites. Nova is eager to learn more about Nazim's interests and pre

In [19]:
# ===============================================
# Part 2: JSON Schema Classification & Information Extraction (Fixed)
# ===============================================

def extract_info(chat_text):
    """
    Given a chat text, extract structured info according to JSON schema
    Returns a dict with extracted fields; missing fields as null
    """
    prompt = f"""
    Extract the following details from the user chat in JSON format:
    {json.dumps(json_schema, indent=2)}

    If a field is missing, return it as null.

    Chat Text:
    \"\"\"{chat_text}\"\"\"
    """

    # Pass the prompt directly as a string
    response = llm_extractor.invoke(prompt).content

    # Try parsing JSON
    try:
        return json.loads(response)
    except:
        return {"raw_output": response}

# Define sample chat messages
sample_chats = [
    "Hello, I'm Sara from Bangalore. Email: sara123@gmail.com, phone: 9123456789. Age 28.",
    "Hi, this is John from Mumbai. Age 32. Email: john_doe@hotmail.com.",
    "My name is Aditi. Phone: 9876543210."
]

# Extract information from sample chats
for i, chat in enumerate(sample_chats, 1):
    extracted = extract_info(chat)
    print(f"\n--- Sample Chat {i} ---")
    print("Chat Text:")
    print(chat)
    print("Extracted JSON:")
    print(json.dumps(extracted, indent=2))




--- Sample Chat 1 ---
Chat Text:
Hello, I'm Sara from Bangalore. Email: sara123@gmail.com, phone: 9123456789. Age 28.
Extracted JSON:
{
  "raw_output": "To extract the details from the user chat in JSON format, we'll write a Python script. We'll use a dictionary to store the user details and then convert it to JSON.\n\n```python\nimport re\nimport json\n\n# regular expressions to extract the data\nemail_pattern = r'email: (\\S+)'\nphone_pattern = r'phone: (\\d+)'\nlocation_pattern = r'from (\\S+)'  # using 'from' instead of 'from ' to include cases like 'from Bangalore'\nage_pattern = r'Age (\\d+)'\n\n# user chat\nchat = \"\"\"Hello, I'm Sara from Bangalore. Email: sara123@gmail.com, phone: 9123456789. Age 28.\"\"\"\n\n# extract data from the chat\ndef extract_data(chat):\n    data = {\n        \"name\": None,  # assuming name is the username, so we're extracting it as 'Sara' from 'I'm Sara'\n        \"email\": None,\n        \"phone\": None,\n        \"location\": None,\n        \"ag

In [20]:
# ===============================================
# Conversation Management & JSON Extraction using Groq API
# (Streamlit version with LangChain)
# ===============================================

%%writefile app.py
import streamlit as st
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# ------------------------------
# 1️⃣ Set Groq / OpenAI API credentials
# ------------------------------
from google.colab import userdata
api_key = "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"
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# ------------------------------
# 2️⃣ Initialize LLMs
# ------------------------------
llm = ChatOpenAI(model="llama-3.1-8b-instant", temperature=0.0)
summarizer_llm = ChatOpenAI(model="llama-3.1-8b-instant", temperature=0.0)
llm_extractor = ChatOpenAI(model="llama-3.1-8b-instant", temperature=0.0)

# ------------------------------
# 3️⃣ Conversation memory setup (per session_id)
# ------------------------------
if "store" not in st.session_state:
    st.session_state.store = {}

if "message_counter" not in st.session_state:
    st.session_state.message_counter = {}

def get_history(session_id: str) -> ChatMessageHistory:
    if session_id not in st.session_state.store:
        st.session_state.store[session_id] = ChatMessageHistory()
    return st.session_state.store[session_id]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly assistant. Keep track of details for this conversation."),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])

conversation_chain = prompt | llm

conversation = RunnableWithMessageHistory(
    conversation_chain,
    get_history,
    input_messages_key="input",
    history_messages_key="history"
)

# ------------------------------
# 4️⃣ Periodic summarization
# ------------------------------
PERIODIC_K = 3  # summarize every 3 messages

def summarize_history(session_id: str):
    history = get_history(session_id).messages
    if not history:
        return "Nothing has been discussed yet.\nWe are in a new conversation."
    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", "Summarize the following conversation so far."),
        ("human", "{history}")
    ])
    chain = summary_prompt | summarizer_llm
    return chain.invoke("\n".join([f"{m.type}: {m.content}" for m in history])).content

# ------------------------------
# 5️⃣ JSON extraction setup
# ------------------------------
json_schema = {
    "name": "string",
    "email": "string",
    "phone": "string",
    "location": "string",
    "age": "integer"
}

def extract_info(chat_text):
    prompt = f"""
    Extract the following details from the user chat in JSON format only and don't give python script or any other way:
    {json.dumps(json_schema, indent=2)}
    If a field is missing, return it as null.

    Chat Text:
    \"\"\"{chat_text}\"\"\"
    """
    response = llm_extractor.invoke(prompt).content
    try:
        return json.loads(response)
    except:
        return {"raw_output": response}

# ------------------------------
# 6️⃣ Streamlit UI
# ------------------------------
st.title("AI Chat + JSON Extractor")

session_id = st.text_input("Enter your session ID:", "user123")
user_input = st.text_input("Type your message:")

if st.button("Send") and user_input:
    # Initialize counter for this session
    if session_id not in st.session_state.message_counter:
        st.session_state.message_counter[session_id] = 0

    # Determine if user wants JSON extraction
    json_mode = "json" in user_input.lower() or "extract" in user_input.lower()

    # Run conversation
    conversation.invoke({"input": user_input}, config={"configurable": {"session_id": session_id}})
    st.session_state.message_counter[session_id] += 1

    # Handle JSON mode
    if json_mode:
        extracted_info = extract_info(user_input)
        st.subheader("Extracted JSON Info:")
        st.json(extracted_info)

    # Show AI reply
    last_ai_message = get_history(session_id).messages[-1].content
    st.subheader("AI Reply:")
    st.write(last_ai_message)

    # Periodic summary
    if st.session_state.message_counter[session_id] % PERIODIC_K == 0:
        summary = summarize_history(session_id)
    elif len(get_history(session_id).messages) == 0:
        summary = "Nothing has been discussed yet.\nWe are in a new conversation."
    else:
        summary = "Conversation in progress. Summary will appear soon."

    # ------------------------------
    # Collapsible Conversation Track
    # ------------------------------
    with st.expander("Conversation Track"):
        st.write(summary)

    # ------------------------------
    # Collapsible Full Conversation History
    # ------------------------------
    with st.expander("Full Conversation History"):
        history_text = "\n".join([f"{m.type}: {m.content}" for m in get_history(session_id).messages])
        st.text(history_text)


Overwriting app.py


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("336OOIhxlnjHOZ6gIX3SoT8Jiq4_6zDcAtbp6nQ6Cxa5qJfzW")
ngrok.kill()
public_url = ngrok.connect(8501)
print("🚀 Streamlit Public URL:", public_url)
!streamlit run app.py &


🚀 Streamlit Public URL: NgrokTunnel: "https://unauctioned-millie-preternaturally.ngrok-free.dev" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.102.108:8501

